# TODO
- ładnie opisać co się dzieje
- ładnie na funkcje podzielić i refactor ogólnie
- obsługa inputu
- obsługa plakatów <3

In [1]:
import numpy as np
import pandas as pd
import requests
import itertools

## Move API

In [17]:
api_key = '63a56440'
url = f'http://www.omdbapi.com/?apikey={api_key}'
params = {"t":"", "plot": "short", "r":"json"}

In [18]:
def fetch_movies(movies):
    movies_data = {}
    for movie in movies:
        params["t"] = movie
        response = requests.get(url, params=params).json()
        movies_data[movie] = response
    return movies_data

## Get input from user

In [36]:
value = None
movies = []

print("Please enter a movie title to compare. When you finish press enter.\n")

while value != '':
    value = input()
    if value != '':
        movies.append(value)

movies_data = fetch_movies(movies)

Please enter a movie title to compare. When you finish press enter.

dfalsodufhao



In [ ]:
criteria = list(movies_data[movies[0]].keys())

for index, criterium in enumerate(criteria[1:]):
    print(f'{index + 1}. {criterium}')

value = None
chosen_criteria = []

print("\nPlease enter a criteria number to compare. When you finish press enter.\n")

while value != '':
    value = input()
    if value != '':
        chosen_criteria.append(criteria[int(value)])

1. Error

Please enter a criteria number to compare. When you finish press enter.



## Create pairs to compare

In [35]:
movies_pairs = list(itertools.combinations(movies, 2))
criteria_pairs = list(itertools.combinations(chosen_criteria, 2))

print("Movies pairs: ", movies_pairs)
print("\nCriteria pairs: ", criteria_pairs)

Movies pairs:  []

Criteria pairs:  []


## User rating movies

In [23]:
movies_matrices = [pd.DataFrame(columns=movies, index=movies) for _ in range(len(chosen_criteria))]
criteria_matrices = pd.DataFrame(columns=chosen_criteria, index=chosen_criteria)


for i, criterium in enumerate(chosen_criteria):
    for pair in movies_pairs:
        matrix = movies_matrices[i]
        np.fill_diagonal(matrix.values, 1)
        
        val =  float(input(f'\nCriterium: {criterium}\n    {pair[0]}: {movies_data[pair[0]][criterium]}\n    {pair[1]}: {movies_data[pair[1]][criterium]}\n Compare first to second:\n'))
        matrix[pair[0]][pair[1]] = val
        matrix[pair[1]][pair[0]] = 1/val



Criterium: Actors
    unforgiven: Clint Eastwood, Gene Hackman, Morgan Freeman
    interstellar: Matthew McConaughey, Anne Hathaway, Jessica Chastain
 Compare first to second:
10

Criterium: Actors
    unforgiven: Clint Eastwood, Gene Hackman, Morgan Freeman
    django: Franco Nero, José Canalejas, José Bódalo
 Compare first to second:
10

Criterium: Actors
    interstellar: Matthew McConaughey, Anne Hathaway, Jessica Chastain
    django: Franco Nero, José Canalejas, José Bódalo
 Compare first to second:
2

Criterium: Type
    unforgiven: movie
    interstellar: movie
 Compare first to second:
1

Criterium: Type
    unforgiven: movie
    django: movie
 Compare first to second:
1

Criterium: Type
    interstellar: movie
    django: movie
 Compare first to second:
1


## User rating criteria priority

In [24]:
for pair in criteria_pairs:
    np.fill_diagonal(criteria_matrices.values, 1) 

    val =  float(input(f'Rate importance: {pair[0]} to {pair[1]}:\n'))
    criteria_matrices[pair[0]][pair[1]] = val
    criteria_matrices[pair[1]][pair[0]] = 1/val

Rate importance: Actors to Type:
10


## EVM ranking method

In [25]:
def count_evm(comparison_matrix):
    matrix = comparison_matrix.to_numpy(dtype=float)
    values, vectors = np.linalg.eigh(matrix)
    max_index = np.argmax(values)
    v = np.abs(vectors[:, max_index])
    return v / np.sum(v)

In [26]:
priorities = pd.DataFrame(columns=movies, index=chosen_criteria)
criteria_importance = pd.DataFrame(columns=chosen_criteria)

In [27]:
# count priorities and importance
# multiply priorities * importance
# sum up
criteria_matrices

,Actors,Type
Actors,1,0.1
Type,10.0,1


In [28]:
criteria_priorities = pd.DataFrame(columns=chosen_criteria)
criteria_priorities.loc[0] = count_evm(criteria_matrices)
criteria_priorities = criteria_priorities.rename(index={0: "priority"})

comparison_priorities = pd.DataFrame(columns=chosen_criteria, index=movies)
for i, criterium in enumerate(chosen_criteria):
    matrix = movies_matrices[i]
    comparison_priorities[criterium] = count_evm(matrix)   

In [29]:
comparison_priorities

,Actors,Type
unforgiven,0.397181,0.333333
interstellar,0.301410,0.333333
django,0.301410,0.333333


In [30]:
criteria_priorities

,Actors,Type
priority,0.5,0.5


In [31]:
for criterium in chosen_criteria:  
    comparison_priorities[criterium] = comparison_priorities[criterium].values * criteria_priorities[criterium].values
comparison_priorities

,Actors,Type
unforgiven,0.198590,0.166667
interstellar,0.150705,0.166667
django,0.150705,0.166667


In [32]:
comparison_priorities["Goal"] = comparison_priorities.sum(axis=1)
comparison_priorities

,Actors,Type,Goal
unforgiven,0.198590,0.166667,0.365257
interstellar,0.150705,0.166667,0.317371
django,0.150705,0.166667,0.317371


# ---------------------------------------------------------------------------------------------------------------

dataframe = pd.read_csv("piwo.csv", index_col=0, sep=' ')

In [ ]:
dataframe

In [ ]:
dataframe.columns.values.tolist()

In [ ]:
dataframe[["Koszt", "Procent"]]

In [ ]:
dataframe["Koszt"]

In [ ]:
def get_columns_names(dataframe):
    return dataframe.columns.values.tolist()

In [ ]:
print(get_columns_names(dataframe))

In [ ]:
def filter_dataframe_columns(dataframe, columns_to_stay):
    dataframe_columns_names = get_columns_names(dataframe)
    columns_to_drop = set(dataframe_columns_names) - set(columns_to_stay)
    return dataframe.drop(columns=columns_to_drop)

In [ ]:
only_good_values = filter_dataframe_columns(dataframe, ["Koszt", "Procent", "Smak"])
only_good_values

In [ ]:
def count_evm(comparison_matrix):
    matrix = comparison_matrix.to_numpy(dtype=float)
    values, vectors = np.linalg.eigh(matrix)
    max_index = np.argmax(values)
    v = np.abs(vectors[:, max_index])
    return v / np.sum(v)

In [ ]:
comp_matrix = np.array([[1,2, 9],[1/2,1, 2], [1/9, 1/2, 1]])
print(comp_matrix)
w, v = count_eigenvalue(comp_matrix)
print(w)
print(v)

In [ ]:
def get_final_ranking(dataframe, weights):
    data = dataframe.to_numpy()
    return np.sum(data*weights, axis=-1)

In [ ]:
print(make_ranking(only_good_values, w))

In [ ]:
def apply_ranking_to_dataframe(dataframe, weigth):
    result_dataframe = dataframe.copy(deep=True)
    ranking = get_final_ranking(dataframe, weigth)
    result_dataframe["Wyniki"] = ranking.tolist()
    result_dataframe = result_dataframe.sort_values("Wyniki", ascending=False)
    return result_dataframe

In [ ]:
result_dataframe = apply_ranking_to_dataframe(only_good_values, w)
result_dataframe